In [2]:
import os
import numpy as np
import torch

device = 'cpu'
block_size = 1023
batch_size = 2
dataset = ''

# poor man's data loader
data_dir = dataset
train_data = np.memmap(os.path.join(data_dir, 'train.bin'), dtype=np.uint8, mode='r')
val_data = np.memmap(os.path.join(data_dir, 'val.bin'), dtype=np.uint8, mode='r')
def get_batch(split):
    data = train_data if split == 'train' else val_data
    # ix = torch.randint(len(data) - block_size, (batch_size,))
    # Ensure the starting index is a multiple of block_size
    ix = torch.randint(0, len(data) // (block_size + 1), (batch_size,)) * (block_size + 1)
    x = torch.stack([torch.from_numpy((data[i:i+block_size]).astype(np.int64)) for i in ix])
    y = torch.stack([torch.from_numpy((data[i+1:i+1+block_size]).astype(np.int64)) for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

In [3]:
print(len(train_data))
print(len(val_data) / 1024)

8394215424
82803.0


In [4]:
import pickle
import os


# Usage
# meta_file_path = os.path.join(os.path.dirname(__file__), 'meta.pkl')
file_path = 'meta.pkl'
with open(file_path, 'rb') as f:
    data = pickle.load(f)
print(data)

itos = data['itos']

def decode(l):
    return ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

{'vocab_size': 32, 'itos': {0: ' ', 1: '#', 2: '+', 3: '-', 4: '.', 5: '0', 6: '1', 7: '2', 8: '3', 9: '4', 10: '5', 11: '6', 12: '7', 13: '8', 14: '9', 15: ';', 16: '=', 17: 'B', 18: 'K', 19: 'N', 20: 'O', 21: 'Q', 22: 'R', 23: 'a', 24: 'b', 25: 'c', 26: 'd', 27: 'e', 28: 'f', 29: 'g', 30: 'h', 31: 'x'}, 'stoi': {' ': 0, '#': 1, '+': 2, '-': 3, '.': 4, '0': 5, '1': 6, '2': 7, '3': 8, '4': 9, '5': 10, '6': 11, '7': 12, '8': 13, '9': 14, ';': 15, '=': 16, 'B': 17, 'K': 18, 'N': 19, 'O': 20, 'Q': 21, 'R': 22, 'a': 23, 'b': 24, 'c': 25, 'd': 26, 'e': 27, 'f': 28, 'g': 29, 'h': 30, 'x': 31}}


In [5]:
X, Y = get_batch('val')
print("Batch")
print(X)
print("y")
print(Y)

Batch
tensor([[15,  6,  4,  ...,  0, 17, 29],
        [15,  6,  4,  ...,  6,  2,  0]])
y
tensor([[ 6,  4, 27,  ..., 17, 29,  7],
        [ 6,  4, 27,  ...,  2,  0,  7]])


In [6]:
print(decode(X[0].tolist()))
print(decode(X[1].tolist()))
print(decode(Y[0].tolist()))
print(decode(Y[1].tolist()))

;1.e3 d5 2.d3 Bg4 3.c3 Bxd1 4.Kxd1 Nf6 5.Kc2 e5 6.Kb3 e4 7.Kc2 d4 8.dxe4 dxe3 9.Bxe3 Nxe4 10.Nf3 f5 11.Bd3 f4 12.Bxe4 fxe3 13.fxe3 Qe7 14.Nfd2 Nc6 15.Re1 Ne5 16.Bxb7 Rb8 17.Ba6 Qc5 18.b4 Qb5 19.Bxb5+ Rxb5 20.a4 Rb8 21.Na3 Rd8 22.Nac4 Nxc4 23.Nxc4 Bd6 24.Nxd6+ cxd6 25.c4 O-O 26.b5 Rf2+ 27.Kd3 Rdf8 28.Re2 R2f6 29.e4 d5 30.exd5 R8f7 31.Re3 Rf2 32.Re2 R2f5 33.Rae1 Rf4 34.Re4 Rf8 35.R1e2 R4f6 36.Kd4 g6 37.c5 Kg7 38.d6 R6f7 39.c6 Rd8 40.a5 Rxd6+ 41.Kc5 Rff6 42.b6 axb6+ 43.Kb5 bxa5 44.Kxa5 Rxc6 45.Re7+ Kh6 46.R2e5 Ra6+ 47.Kb5 Rfb6+ 48.Kc5 Rc6+ 49.Kd5 Rcb6 50.Ke4 Ra4+ 51.Kf3 Rbb4 52.Ke2 Rb3 53.Kd2 Ra2+ 54.Kc1 Re3 55.Rb5 Rf2 56.Rb2;1.e4 d6 2.d4 g6 3.Be3 Bg7 4.Bd3 c5 5.c3 cxd4 6.cxd4 e5 7.dxe5 dxe5 8.Ne2 Bg4 9.f3 Be6 10.O-O Ne7 11.Nbc3 O-O 12.a3 Nbc6 13.b4 Bb3 14.Qb1 Nd4 15.Nb5 Nxe2+ 16.Bxe2 Ba4 17.Qb2 Bxb5 18.Bxb5 Qd3 19.Bxd3;1.e4 a6 2.d4 b5 3.Nf3 f6 4.Nc3 g5 5.b4 e6 6.Bb2 Bxb4 7.d5 e5 8.g4 d6 9.h3 h5 10.Bd3 hxg4 11.Nh2 gxh3 12.O-O Bxc3 13.Bxc3 Ne7 14.Qf3 Ng6 15.Ng4 Nf4 16.Ne3 h2+ 17.Kh1 Bh3 18